## Introduction
The goal of this project is to predict geospatialy resolved wildfire ignition risk in California with a neural network trained on historical weather and wildfire data. Two approaches were attempted and scaled. The most effective approach was a parallel long short term memory network (pLSTM). This project had six phases. See below for more detail on each part and a link to the relevant notebook.

1. [Feature addition and smoothing](https://github.com/gperdrizet/wildfire/blob/master/notebooks/add_features.ipynb) - Added min, mean and max features for each weather variable and smoothed the whole dataset with a daily average.
2. [Fullly stratified sampling](https://github.com/gperdrizet/wildfire/blob/master/notebooks/recursive_sampling.ipynb) - Dataset was split into fully stratified blocks of ~500,000 observations.
3. [Small scale MLP](https://github.com/gperdrizet/wildfire/blob/master/notebooks/keras_MLP_skopt.ipynb) - The stratified samples were used to optimize and test a 'deep' neural network architecture using a binary 'fire/no fire' paradigm.
4. [Scaled MLP]() - Insights gained from the small scale MLP were applied to the entire dataset.
5. [Single LSTM](https://github.com/gperdrizet/wildfire/blob/master/notebooks/keras_stateless_LSTM_skopt_one_spatial_bin.ipynb) - A simple, long short term memory based network architecture was optimized and tested on the full 22 years of data in just one geospatial bin.
6. [Parallel LSTM](https://github.com/gperdrizet/wildfire/blob/master/notebooks/keras_parallel_LSTM_410input.ipynb) A parallel LSTM network consisting of 410 inputs, on for each geospatial bin, was trained on all 22 years of data.

## 1. Feature addition and smoothing
The motivation to add min, max and mean features and then smooth by daily averaging was to 1) reduce the size of the data set and 2) make the resolution of the weather data more closely match that of the fire data (i.e. day of fire discovery)

Min, max and mean features were calculated with a sliding window of 24 hr. The calculated values were added at the right edge of the window. This was done to reflect the temporal nature of the data - the conditions which led to a fire must have occurred on and/or before the day that the fire was discovered. The results of feature addition for some representative weather variables are shown below.

<p  style="text-align: center;"><b>Comparison of min, max and mean features to raw data</b></p>

![Example of three weather variables showing the effect of adding min, max and mean](https://github.com/gperdrizet/wildfire/blob/master/figures/min_max_mean_added.png?raw=true)

The second step in the data preparation was compression via the daily mean. The data was compressed to match the resolution of the weather data to that of the fire data. The weather data has a periodicity of 3 hours, while most of the fire data is no more precise than 'discovery day'. The result is shown below.

<p  style="text-align: center;"><b>Data after daily average smoothing</b></p>

![Example of three weather variables showing the effect of smoothing via daily average](https://github.com/gperdrizet/wildfire/blob/master/figures/smoothed_data.png?raw=true)

## 2. Fully stratified sampling 


The motivation behind this phase of the project was two fold: 1) reduce the size of the dataset by sampling. 2) ensure that the distribution of all variables is similar across the samples.

Early on, it was observed that some features which should likely be of high importance ranked highly in some training runs but not in others (ex: month - California has a clear fire season). Ultimately, the best explanation was lucky vs unlucky train, test splits. If a given variable's distribution in the training set does not match the distribution in the test or validation sets it will have poor predictive power. Shown below is a comparison of several representative weather variables between two fully stratified samples.

<p  style="text-align: center;"><b>Comparison of weather feature distributions in fully stratified samples</b></p>

![Comparison of representative weather variable distributions between fully stratified samples](https://github.com/gperdrizet/wildfire/blob/master/figures/stratified_sample_comparison.png?raw=true)

A recursive strategy was employed to accomplish sampling. The dataset was split randomly and the two sample Kolmogorov–Smirnov was conducted on all variables between the two haves. If none of the distributions were found to be significantly different (p >= 0.3), the split was accepted for further recursion. The base condition was n <= 500,000.

## 3. Small scale multilayer perceptron
The first neural network constructed was a feed forward multilayer perceptron. The network was trained and tested using different fully stratified sample for training, validation and testing. Scikit-optimize was used to conduct gaussian optimization of model hyperparameters. The parameters optimized were:
1. Learning rate
2. Number of hidden layers
3. Units per hidden layer
4. Dropout rate in a single distal dropout layer
5. L2 regularization lambda coefficient
6. The output class weighting

After hyperparameter optimization, the final model was trained using the winning hyperparameters. A larger batch size and total fraction of data was used along with some manual hyperparameter curation. The resulting learning curves are shown below.

<p  style="text-align: center;"><b>Learning curves for small scale MLP model</b></p>

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/simple_MLP_learning_curves.png?raw=true)

To visually asses model performance, a single geospatial bin was recovered from the original data and used for prediction. The results were plotted against true ignition at the same location. A representative sample of the results are shown below.

<p  style="text-align: center;"><b>Fires predicted from training data: small scale MLP model</b></p>

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/simple_MLP_training_predictions.png?raw=true)

Unfortunately, as can be seen from the results above, the model struggled even to memorize the training data. This is maybe not surprising, because the MLP model has no sense of the data's order. The right most panel in the figure above shows roughly 2 years worth of predictions and it is visually obvious that the seasonality of fires in California has been lost - the 'days' on the x-axis are not necessarily in chronological order. This realization directly led to the use of a short long term memory network in part 5.

Not surprisingly the predictions on the test data are not any better.

<p  style="text-align: center;"><b>Fires predicted from test data: small scale MLP model</b></p>

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/simple_MLP_test_predictions.png?raw=true)

## 4. Scaled multilayer perceptron

Note: due to concerns about the performance of the small scale MLP model in part 3 and compute time, this section was omitted pending improvements to the small scale model.

## 5. Single geospatial long short term memory network
The single input LSTM network works on the weather and fire time series from just one location in California. It consists of one LSTM layer, then one or more fully connected layers and a single output. It was optimized using Scikit-Optimize. The hyperparameter investigated were:

1. Learning rate
2. Size of the sliding time window used to construct samples
3. The number of LSTM units
4. The number of fully connected layers after the LSTM
5. The number of units in each fully connected layer
6. The L2 regularization lambda term in the fully connected layers
7. The output class weighting

After hyperparameter optimization, the final model was trained using the winning hyperparameters with some manual curation. The resulting learning curves are shown below.

<p  style="text-align: center;"><b>Learning curves: single bin LSTM model</b></p>

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/single_stateless_LSTM_learning_curves.png?raw=true)

To visually asses model performance, true ignitions were plotted against predicted ignitions. Some representative time windows are shown below.

<p  style="text-align: center;"><b>Fires predicted from training data: single bin LSTM model</b></p>

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/single_stateless_LSTM_training_predictions.png?raw=true)

The fourth panel above clearly shows that the order of days has been preserved and that the model is capable of recognizing fire season. However, it still struggles to predict individual fires accurately.

Seen below are the results of predictions on the test dataset. They are encouragingly similar to the predictions on the training set - on the scale of weeks, the model can identify high fire risk conditions, but it is unable to predict specific ignition days.

![Learning curves for small scale MLP model](https://github.com/gperdrizet/wildfire/blob/master/figures/single_stateless_LSTM_testing_predictions.png?raw=true)

## 6. Parallel geospatial long short term memory network
The parallel geospatial LSTM network is the most complex and original of the models presented so far. It is an exotic architecture tailored to the problem at hand using the Keras functional API. It consists of 410 input LSTM layers, each corresponding to one geospatial bin in California. The output from each LSTM is then fed through a merge layer and into a sequence of fully connected hidden layers. The final output is a fully connected layer with 410 units - one for the fire risk in each of the 410 geospatial bins.

![Parallel LSTM learning curves](https://github.com/gperdrizet/wildfire/blob/master/figures/parallel_LSTM_learning_curves.png?raw=true)